In [ ]:
import pandas as pd
import cbsodata

print(cbsodata.__version__)

table_id = '83765NED' #wijken en buurten 2017

In [ ]:
# Downloading table list
# toc = pd.DataFrame(cbsodata.get_table_list())
tableInfos = cbsodata.get_meta(table_id=table_id, name='TableInfos')
print(table_id, ':', tableInfos[0]['Summary'])

Select all the information on my Municipality

In [ ]:
filters="indexof(WijkenEnBuurten, 'BU0809') eq 0"
# filters="not indexof(WijkenEnBuurten, 'GM0809') eq 0"
dimensions = [
        'WijkenEnBuurten',
        'Codering_3',
        'AantalInwoners_5',
      #  'GemiddeldeWoningwaarde_35',
     #   'GemiddeldElektriciteitsverbruikTotaal_47',
     #   'GemiddeldAardgasverbruikTotaal_55',
        'BedrijfsvestigingenTotaal_78']

wijkenEnBuurten =  pd.DataFrame(
        cbsodata.get_data(
                table_id=table_id, 
                filters=filters,
                select=dimensions))

What looks to be a good criterium to see if we are dealing with an industrial zone, is to compare the number of companies to the number of residents.

In [ ]:
def isIndustrialZone(bedrijfsvestigingenTotaal, AantalInwoners):
    # A first derivation of what could be an industrial zone
    return 2 * bedrijfsvestigingenTotaal > AantalInwoners

In [ ]:
wijkenEnBuurten['industial_zone'] = isIndustrialZone(wijkenEnBuurten['BedrijfsvestigingenTotaal_78'], wijkenEnBuurten['AantalInwoners_5'])
wijkenEnBuurten[wijkenEnBuurten['industial_zone'] == True]

Add to the 'decentrale opwek' data the new information on wheter we are dealing with an industrial zone or not.
//TODO:


In [75]:
def makebuurt(code): #taken from the python document of Greg :-)
    if len(code) == 6:
        return 'BU00' + code
    elif len(code) == 7:
        return 'BU0' + code
    else:
        return 'BU' + code

In [76]:
datafilename = './data/Enexis_decentrale_opwek_kv_(zon_pv)_01012020.csv'
df = pd.read_csv(datafilename, sep=';', encoding= 'UTF-8')
df['Buurtcode']= df['CBS Buurtcode'].astype(str).apply(makebuurt)
df.head()

,Peildatum,Netbeheerder,Provincie,Gemeente,CBS Buurt,CBS Buurtcode,Aantal aansluitingen in CBS-buurt,Aantal aansluitingen met opwekinstallatie,Opgesteld vermogen,Buurtcode
0,202001.0,Enexis,Groningen,Appingedam,Appingedam-Centrum,30000.0,1.480,155.0,507.000,BU030000.0
1,202001.0,Enexis,Groningen,Appingedam,Appingedam-West,30001.0,1.331,479.0,1.685,BU030001.0
2,202001.0,Enexis,Groningen,Appingedam,Appingedam-Oost,30002.0,2.826,439.0,1.616,BU030002.0
3,202001.0,Enexis,Groningen,Appingedam,Verspreide huizen Damsterdiep en Eemskanaal,30007.0,230.000,97.0,405.000,BU030007.0
4,202001.0,Enexis,Groningen,Appingedam,Verspreide huizen ten zuiden van Eemskanaal,30008.0,102.000,21.0,61.000,BU030008.0
